In [0]:
%pip install databricks-vectorsearch
%pip install -U mlflow
%restart_python

In [0]:
import mlflow
from mlflow.models import ModelConfig

cfg = ModelConfig(development_config="config.yml")
endpoint_name=cfg.get("retriever").get("vs_endpoint")
vs_index=cfg.get("retriever").get("vs_index")
source_table = cfg.get("retriever").get("vs_source")
endpoint_name, vs_index, source_table

In [0]:
from databricks.vector_search.client import VectorSearchClient

client = VectorSearchClient()
try:
    client.create_endpoint(name=endpoint_name)
except Exception as e:
    if 'already exists' in str(e):
        print(f"Endpoint {endpoint_name} already exists.")
    else:
        raise

In [0]:
index = client.create_delta_sync_index(
  endpoint_name=endpoint_name,
  source_table_name=source_table,
  index_name=vs_index,
  columns_to_sync=["smiles", "zinc_id", "mwt", "logp", 
                   "reactive", "purchasable", "tranche_name", "SPS", "FractionCSP3", "RingCount", "ecfp"],
  pipeline_type="TRIGGERED",
  primary_key="zinc_id",
  embedding_dimension=1024,
  embedding_vector_column="ecfp"
)